<a href="https://colab.research.google.com/github/YamnaIsrail/recipe-gpt2-fine-tuning/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling, TextDataset


In [2]:

# Load Pre-trained Model and Tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:

# Load and Preprocess Data
train_path = "mydata.csv"  # Update the file name
df = pd.read_csv(train_path)
questions = df['Question'].tolist()
answers = df['Answer'].tolist()


In [5]:

# Combine questions and answers for training
texts = [f"Q: {q}\nA: {a}" for q, a in zip(questions, answers)]

# Tokenize the combined texts
tokenized_texts = [tokenizer.encode(text, return_tensors="pt").squeeze() for text in texts]
input_ids = torch.cat(tokenized_texts)

# Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [6]:

# Training Configuration
training_args = {
    "output_dir": "./qa_finetuned",
    "overwrite_output_dir": True,
    "num_train_epochs": 1,
    "per_device_train_batch_size": 4,
    "save_steps": 10_000,
    "save_total_limit": 2
}

# Initialize Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, no_deprecation_warning=True)


In [7]:

# Training Loop
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128
)

train_dataloader = DataLoader(
    train_dataset,
    collate_fn=data_collator,
    batch_size=training_args["per_device_train_batch_size"],
    shuffle=True
)

for epoch in range(training_args["num_train_epochs"]):
    for step, batch in enumerate(train_dataloader):
        input_ids = batch["input_ids"].to(model.device)
        labels = batch["labels"].to(model.device)

        outputs = model(input_ids, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 1000 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item()}")


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Epoch 0, Step 0, Loss: 3.462501287460327


In [8]:

# Save Fine-tuned Model
model.save_pretrained(training_args["output_dir"])
tokenizer.save_pretrained(training_args["output_dir"])

# Load Fine-tuned Model for Chatbot
model = GPT2LMHeadModel.from_pretrained(training_args["output_dir"])
tokenizer = GPT2Tokenizer.from_pretrained(training_args["output_dir"])


In [9]:

# Chatbot Interaction
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Exiting chatbot.")
        break

    # Use a simple prompt structure
    prompt = f"You: {user_input}\nBot:"
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Ensure attention_mask is set
    attention_mask = torch.ones(input_ids.shape, device=model.device)


You: i am feeling down
You: i am exhausted
You: exit
Exiting chatbot.


In [10]:

    # Set pad_token_id to eos_token_id for open-end generation
    model.config.pad_token_id = model.config.eos_token_id

    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,
        num_beams=5,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95
    )

    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Bot: {generated_response}")



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Bot: You: i am exhausted
Bot: you have a hard time sleeping, and you feel like you are in a mental state. How can you deal with this?
How can I cope with the loss of a loved one?


What is the most important thing you can do to overcome the fear of death?
